# SQL 작성

In [1]:
import sys
import os
import pandas as pd
import cv2
from glob import glob
import xml.etree.ElementTree as ET
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE robo_baby_box_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        label VARCHAR(10) NOT NULL,
        box_xtl INT(10) NOT NULL,
        box_ytl INT(10) NOT NULL,
        box_xbr INT(10) NOT NULL,
        box_ybr INT(10) NOT NULL
        );''')
query2 = (f'''INSERT INTO robo_baby_box_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, label, box_xtl, box_ytl,
          box_xbr, box_ybr
          ) VALUES (%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s, %s)''')


In [4]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [5]:
label_path = []
label_name= []

# label 경로
label_path = 'Z:\\robo_baby\\data\\RGB\\labeldata'
image_path = 'Z:\\robo_baby\\data\\RGB\\rawdata'

value_list1 = []
error_list = []
count = 0

labels = os.listdir(label_path)
for num, label_id in enumerate(labels):
    total_cnt = len(labels)
    color_space = 'RGB'
    label_files = os.listdir(os.path.join(label_path,label_id))
    
    for label_file in label_files:
        
        label_dir = os.path.join(label_path,label_id,label_file)


        label_name, label_format = os.path.splitext(os.path.basename(label_file))
        label_format = label_format.replace('.','')

        tree = ET.parse(label_dir)
        root = tree.getroot()

        img_dir = label_dir.replace('label','raw').replace('.xml','.jpg')



        img_file = os.path.basename(img_dir)
        img_name = label_name
        img_format = 'jpg'

        if os.path.exists(img_dir) and os.path.getsize(img_dir) != 0:
            img = cv2.imread(img_dir)            
        else:
            error_list.append(img_dir)
            continue
        img_w = img.shape[0]
        img_h = img.shape[1]

        img_dir = img_dir.replace('Z:','DataBase')
        label_dir = label_dir.replace('Z:','DataBase')

        for i in root.findall('object'):
            label = i.find('name').text
            boxes = i.findall('bndbox')
            for box in boxes:
                
                box_xtl = int(box.find('xmin').text)
                box_ytl = int(box.find('ymin').text)
                box_xmax = int(box.find('xmax').text)
                box_ymax = int(box.find('ymax').text)
                box_w = abs(box_xtl - box_xmax)
                box_h = abs(box_ytl - box_ymax)


                values = (img_dir, img_format, img_w, img_h, color_space,
                          label_dir, label_format, label, box_xtl, 
                          box_ytl, box_w, box_h)

                value_list1.append(values)


        # query 실행
        if len(value_list1) >= 500:
            print(value_list1[-1])
            doUT.insert_dataset_values(query2, value_list1)
            value_list1 = []
            print(num, total_cnt)

doUT.insert_dataset_values(query2, value_list1)
value_list1 = []
print('done')

('DataBase\\robo_baby\\data\\RGB\\rawdata\\kids\\kid_275.jpg', 'jpg', 640, 640, 'RGB', 'DataBase\\robo_baby\\data\\RGB\\labeldata\\kids\\kid_275.xml', 'xml', 'adult', 59, 118, 449, 523)
insert 500 done
0 1
('DataBase\\robo_baby\\data\\RGB\\rawdata\\kids\\kid_530.jpg', 'jpg', 640, 640, 'RGB', 'DataBase\\robo_baby\\data\\RGB\\labeldata\\kids\\kid_530.xml', 'xml', 'adult', 403, 196, 238, 428)
insert 500 done
0 1
('DataBase\\robo_baby\\data\\RGB\\rawdata\\kids\\kid_736.jpg', 'jpg', 640, 640, 'RGB', 'DataBase\\robo_baby\\data\\RGB\\labeldata\\kids\\kid_736.xml', 'xml', 'adult', 6, 72, 569, 569)
insert 501 done
0 1
('DataBase\\robo_baby\\data\\RGB\\rawdata\\kids\\kid_1038.jpg', 'jpg', 640, 640, 'RGB', 'DataBase\\robo_baby\\data\\RGB\\labeldata\\kids\\kid_1038.xml', 'xml', 'kids', 185, 148, 244, 416)
insert 500 done
0 1
('DataBase\\robo_baby\\data\\RGB\\rawdata\\kids\\kid_1268.jpg', 'jpg', 640, 640, 'RGB', 'DataBase\\robo_baby\\data\\RGB\\labeldata\\kids\\kid_1268.xml', 'xml', 'kids', 301, 92